In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
import random
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
import urllib.parse

# Excel dosyasını yükleyelim
df = pd.read_excel("matchh.xlsx")

# ChromeDriver kurulumunu başlatalım
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Ürün bağlantılarını saklayacak yeni sütunlar oluşturalım
df["product_link1"] = None
df["product_link2"] = None
df["product_link3"] = None

# Her bir başlık için arama yapacağız
for index, row in df.iterrows():
    product_title = row['Ürün İsmi']
    
    # Arama URL'sini oluştur
    search_url = f"urunaramauzantısı={urllib.parse.quote(product_title)}"
    driver.get(search_url)
    
    # Sayfanın yüklenmesi için rastgele bekleme süresi (2 ila 5 saniye arasında)
    time.sleep(random.uniform(2, 5))
    
    try:
        # Sayfada aşağı kaydırma işlemi
        scroll_count = random.randint(1, 3)  # Rastgele sayıda kaydırma (1 ila 3 arasında)
        for _ in range(scroll_count):
            driver.execute_script("window.scrollBy(0, 500);")  # 500 piksel aşağı kaydır
            time.sleep(random.uniform(1, 3))  # Kaydırmalar arasında da rastgele bekleme
        
        # İlk üç sonucu bul ve bağlantıları al
        results = driver.find_elements(By.CSS_SELECTOR, ".prdct-cntnr-wrppr .p-card-wrppr a")[:3]
        
        # İlk üç bağlantıyı DataFrame'e ekleyelim
        for i, result in enumerate(results):
            product_link = result.get_attribute("href")
            df.at[index, f"product_link{i+1}"] = product_link

    except Exception as e:
        print(f"Ürün bulunamadı veya hata oluştu: {product_title}")
        df.at[index, "product_link1"] = "Link bulunamadı"
        df.at[index, "product_link2"] = "Link bulunamadı"
        df.at[index, "product_link3"] = "Link bulunamadı"
    
    # Her arama arasında rastgele bekleme süresi (3 ila 6 saniye arasında)
    time.sleep(random.uniform(3, 6))

# Tarayıcıyı kapatalım
driver.quit()

# Sonucu yeni dosyaya kaydedelim
output_file_path = "matchh.xlsx"  # Kayıt yeri olarak betiğin olduğu dizini kullanır
df.to_excel(output_file_path, index=False)

print(f"Tamamlandı! Dosya kaydedildi: {output_file_path}")
